In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import MarkerCluster

# File paths for input data
accident_data_path = "Indiana_Accidents_Since_2011_v2.csv"
county_svi_path = "indiana_epl_spl_rpl_with_additional.csv"
tract_svi_path = "Indiana.csv"
tract_shapefile_path = "Indiana.shp"
county_shapefile_path = "indiana_svi_shapefile.shp"
indiana_rails_path = "Railroads_100k_Scale.shp"

# Load data
accident_data = pd.read_csv(accident_data_path)
county_svi = pd.read_csv(county_svi_path)
tract_svi = pd.read_csv(tract_svi_path)
tract_gdf = gpd.read_file(tract_shapefile_path)
county_gdf = gpd.read_file(county_shapefile_path)
rails_gdf = gpd.read_file(indiana_rails_path)

# Ensure CRS matches for geometries
tract_gdf = tract_gdf.to_crs("EPSG:4326")
county_gdf = county_gdf.to_crs("EPSG:4326")
rails_gdf = rails_gdf.to_crs("EPSG:4326")

# Normalize FIPS columns
county_svi['FIPS'] = county_svi['FIPS'].astype(str).str.zfill(5).str.strip()
county_gdf['FIPS'] = county_gdf['FIPS'].astype(str).str.zfill(5).str.strip()

# Merge county-level SVI data
county_gdf = county_gdf.merge(
    county_svi[['FIPS', 'RPL_THEMES', 'LOCATION']],
    on="FIPS",
    how="left"
).rename(columns={'RPL_THEMES': 'County_RPL_THEMES', 'LOCATION': 'County_Location'})

# Normalize GEOID and FIPS for census tracts
tract_gdf['GEOID'] = tract_gdf['GEOID'].astype(str).str.zfill(11).str.strip()
tract_svi['FIPS'] = tract_svi['FIPS'].astype(str).str.zfill(11).str.strip()

# Merge census tract SVI data
tract_gdf = tract_gdf.merge(
    tract_svi[['FIPS', 'RPL_THEMES', 'LOCATION']],
    left_on="GEOID",
    right_on="FIPS",
    how="left"
).rename(columns={'RPL_THEMES': 'Tract_RPL_THEMES', 'LOCATION': 'Tract_Location'})

# Clip SVI scores to [0, 1]
tract_gdf['Tract_RPL_THEMES'] = tract_gdf['Tract_RPL_THEMES'].clip(lower=0, upper=1).fillna(0)

# Transform SVI scores to a scale of 0 to 10
tract_gdf['Tract_RPL_THEMES_Scale10'] = tract_gdf['Tract_RPL_THEMES'] * 10

tract_gdf.to_csv('Indiana_SVI_Scale1-10.csv')

# Perform spatial join to associate rail segments with census tracts
rails_tract_gdf = gpd.sjoin(rails_gdf, tract_gdf, how="left", predicate="intersects")
rails_tract_gdf['Tract_RPL_THEMES_Scale10'] = rails_tract_gdf['Tract_RPL_THEMES'].fillna(0) * 10

# Step 1: Add SVI Score for Accident Locations
geometry = [Point(xy) for xy in zip(accident_data['Longitude'], accident_data['Latitude'])]
accident_gdf = gpd.GeoDataFrame(accident_data, geometry=geometry, crs="EPSG:4326")
accident_gdf = gpd.sjoin(accident_gdf, tract_gdf[['geometry', 'Tract_RPL_THEMES_Scale10']], how="left", predicate="intersects")

# Step 2: Create Folium map
m = folium.Map(location=[40.2672, -86.1349], zoom_start=7)

# Step 3: Add Census Tract-level SVI Choropleth
folium.Choropleth(
    geo_data=tract_gdf,
    data=tract_gdf,
    columns=['GEOID', 'Tract_RPL_THEMES_Scale10'],
    key_on='feature.properties.GEOID',
    fill_color='YlGnBu',
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name="Census Tract SVI Scores (0-10)",
    bins=[0, 2, 4, 6, 8, 10]
).add_to(m)

# Step 4: Add Census Tract Tooltips
folium.GeoJson(
    tract_gdf,
    style_function=lambda x: {
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['GEOID', 'Tract_RPL_THEMES_Scale10', 'Tract_Location'],
        aliases=['Census Tract:', 'SVI Score (0-10):', 'Location:'],
        localize=True
    ),
    name="Census Tract Tooltips"
).add_to(m)

# Step 5: Highlight Rail Segments with SVI > 5
folium.GeoJson(
    rails_tract_gdf,
    style_function=lambda x: {
        'color': 'red' if float(x['properties'].get('Tract_RPL_THEMES_Scale10', 0)) > 5 else 'blue',
        'weight': 2.5 if float(x['properties'].get('Tract_RPL_THEMES_Scale10', 0)) > 5 else 1,
        'fillOpacity': 0.5
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['railroad', 'Tract_RPL_THEMES_Scale10', 'Tract_Location'],
        aliases=['Railroad:', 'Tract SVI Score (0-10):', 'Location:'],
        localize=True
    ),
    name="Highlighted Rail Segments (SVI > 5)"
).add_to(m)

# Step 6: Add Accident Points with SVI Scores
marker_cluster = MarkerCluster().add_to(m)
for _, row in accident_gdf.iterrows():
    popup_info = f"""
        <b>Latitude:</b> {row['Latitude']}<br>
        <b>Longitude:</b> {row['Longitude']}<br>
        <b>Accident Type:</b> {row.get('Accident Type', 'N/A')}<br>
        <b>SVI Score (0-10):</b> {row.get('Tract_RPL_THEMES_Scale10', 'N/A')}<br>
        <b>Location:</b> {row.get('County Name', 'N/A')}
    """
    folium.Marker(
        location=(row['Latitude'], row['Longitude']),
        popup=folium.Popup(popup_info, max_width=300)
    ).add_to(marker_cluster)

# Step 7: Add Layer Control
folium.LayerControl().add_to(m)

# Save the map
output_path = "indiana_svi_accidents_rail_map_scale10.html"
m.save(output_path)
print(f"Map with SVI scores (0-10), accidents, and rail highlights saved to {output_path}")
